
# Load customer service data

In [0]:
from databricks.sdk import WorkspaceClient
import yaml
import os

# Use the workspace client to retrieve information about the current user
w = WorkspaceClient()
user_email = w.current_user.me().display_name
username = user_email.split("@")[0]

#catalog_name = f"{username}_vocareum_com"
catalog_name = "upperbound" # replace with upperbound or name of your catalog
schema_name = "agents"

workspace_id = str(w.get_workspace_id())

# Allows us to reference these values directly in the SQL/Python function creation
dbutils.widgets.text("catalog_name", defaultValue=catalog_name, label="Catalog Name")
dbutils.widgets.text("schema_name", defaultValue=schema_name, label="Schema Name")
dbutils.widgets.text("workspace_id", defaultValue=workspace_id, label="Workspace ID")

spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog_name}.{schema_name}")

In [0]:
import os

data_folder = f"../data/"
csv_files = [f for f in os.listdir(data_folder) if f.endswith('.csv')]
print(csv_files)

In [0]:
import pandas as pd
import os

for csv_file in csv_files:
    table_name = os.path.splitext(csv_file)[0]
    df = pd.read_csv(os.path.join(data_folder, csv_file))
    spark_df = spark.createDataFrame(df)
    spark_df.write.saveAsTable(f"{catalog_name}.{schema_name}.{table_name}", mode='overwrite')